In [1]:
import sys
sys.path.append("../Modules/")
sys.path.append("../pyFiles/")
sys.path.append("../")
import os

import pandas as pd
import numpy as np

# from configobj import ConfigObj
# from configobj import flatten_errors
# from validate import Validator
    
import logging

# from analysisParameters import loadParameters
# from loadDataE1 import loadSessionDict

#fmt = 'logging.Formatter(''%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
fmt = '%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
logging.basicConfig(level=logging.INFO, format=fmt)
logger = logging.getLogger(__name__)

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import os

init_notebook_mode(connected=True)


In [2]:
# sessionDict = projectTrajectories(sessionDict)
import matplotlib.pyplot as plt
%matplotlib notebook

In [3]:
import pickle

file = open('../sessionFiles.pickle', 'rb')
allSessions = pickle.load(file)
file.close()

sessionDict = allSessions[0]

In [4]:
list(sessionDict.keys())

['subID',
 'trialInfo',
 'expConfig',
 'rawExpUnity',
 'rawExpGaze',
 'processedExp',
 'rawCalibUnity',
 'rawCalibGaze',
 'processedCalib',
 'analysisParameters']

In [5]:
# sessionDict['rawCalibUnity']
# sessionDict['rawCalibGaze']

In [6]:

processedExp = sessionDict['processedExp']


In [7]:
processedExp['worldVidTimeSec'] = np.array(processedExp['pupilTimestamp'] - processedExp['pupilTimestamp'].iloc[0])
processedExp['worldVidTimeSec'].tail(10)

7926    36.867783
7927    36.871685
7928    36.871706
7929    36.877798
7930    36.881718
7931    36.882204
7932    36.885655
7933    36.889612
7934    36.892885
7935    36.893618
Name: worldVidTimeSec, dtype: float64

In [8]:
df2 = processedExp

def calcGazeAzEl(row):
    
    x = row[('gaze-normal0','x')] 
    y = row[('gaze-normal0','y')] 
    z = row[('gaze-normal0','z')] 
    
    az = np.rad2deg(np.arctan(np.divide(x,z)))
    el = np.rad2deg(np.arctan(np.divide(y,z)))
    
    x = row[('gaze-normal1','x')] 
    y = row[('gaze-normal1','y')] 
    z = row[('gaze-normal1','z')] 
    
    az2 = np.rad2deg(np.arctan(np.divide(x,z)))
    el2 = np.rad2deg(np.arctan(np.divide(y,z)))

    #Note that the .apply requires you return a single data structure, 
    #so a SINGLE tuple is OK, but the seperate values for az/el are not OK.
    
    return (np.mean([az,az2]),np.mean([el,el2])) 


azEl = df2.apply(lambda arbitraryRowName: calcGazeAzEl(arbitraryRowName),axis=1)

# df2[('gazeInWorldCam','az')],df2[('gazeInWorldCam','el')]= zip(*azEl)
df2[('gazeInWorldCam_az')],df2[('gazeInWorldCam_el')]= zip(*azEl)
df2['deltaT'] = df2['worldVidTimeSec'].diff()


In [9]:
# list(df2)

In [10]:
def calcTargetAzEl(row):
    
    x = row['ballPos','x']
    y = row['ballPos','y']
    z = row['ballPos','z']
    
    az = np.rad2deg(np.arctan(np.divide(x,z)))
    el = np.rad2deg(np.arctan(np.divide(y,z)))
    
    #Note that the .apply requires you return a single data structure, 
    #so a SINGLE tuple is OK, but the seperate values for az/el are not OK.
    
    return (az,el) 

ballAzEl = df2.apply(lambda arbitraryRowName: calcTargetAzEl(arbitraryRowName),axis=1)

df2['ball_az'],df2['ball_el'] = zip(*ballAzEl)
df2['ballVelInWorldCam_az'] = df2['ball_az'].diff() / df2['deltaT']
df2['ballVelInWorldCam_el'] = df2['ball_el'].diff() / df2['deltaT']
df2['ballVelInWorldCam'] = np.abs(np.sqrt(df2['ballVelInWorldCam_az'].diff()**2  + df2['ballVelInWorldCam_el'].diff()**2) / df2['deltaT'])


In [11]:
# yLim = [-100,100]

yLim=[-50,50]
width=800
height=600

startTimeS = df2['worldVidTimeSec'].iloc[0]
endTimeS = df2['worldVidTimeSec'].iloc[-2]

# I googled for, "python find first larger than"
startIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  startTimeS)
endIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  endTimeS)
print('Plotting from index: ' + str(startIndex) + ' to ' + str(endIndex))

colors_idx = ['rgb(0,204,204)','rgb(128,128,128)','rgb(204,0,0)','rgb(102,0,204)']

# from plotly.subplots import make_subplots
# fig = make_subplots(rows=2, cols=1,shared_xaxes=True)

traces = []

###

bEih_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ball_el'][startIndex:endIndex],
    name = 'ball_el',
    marker_color = colors_idx[2],
    mode='markers',
    marker_size = 8,
    opacity = 0.5)

bEih_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ball_az'][startIndex:endIndex],
    name = 'ball_az',
    marker_color = colors_idx[3],
    mode='markers',
    marker_size = 8,
    opacity = 0.5)


traces.append(bEih_el)
traces.append(bEih_az)


eih_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=-df2['gazeInWorldCam_el'][startIndex:endIndex],
    name = 'gazeInWorldCam_el',
    marker_color = colors_idx[0],
    mode='markers',
    marker_size = 5,
    opacity = 1)

eih_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['gazeInWorldCam_az'][startIndex:endIndex],
    name = 'gazeInWorldCam_az',
    marker_color = colors_idx[1],
    mode='markers',
    marker_size = 5,
    opacity = 1)

traces.append(eih_el)
traces.append(eih_az)

layout = dict(
    dragmode= 'pan',
    title='Time Series with Rangeslider',
    width=width,
    height=height,
    yaxis=dict(range=yLim, title='angular position (degrees)'),
    xaxis=dict(
        rangeslider=dict(visible=True))
)


fig = dict(data=traces, layout=layout)

iplot(fig)


Plotting from index: 1 to 7935


## Recalculate GIW using individual eye vectors

In [12]:
rowIn,inputColumn,outputColumn

# Grab gransformation matrix
#headTransform_4x4 = np.reshape(rowIn["viewMat"],[4,4])

mat =[]
for i in range(16):
    mat.append(rowIn["camera"][str(i)])

headTransform_4x4 = np.reshape(mat,[4,4])

# Transpose
headTransform_4x4 = headTransform_4x4.T

# Grab cyc EIH direction
cycEyeInHead_XYZ = rowIn['cycEyeInHead']
# Add a 1 to convert to homogeneous coordinates
cycEyeInHead_XYZW = np.hstack( [cycEyeInHead_XYZ,1])

# Take the dot product!
cycGIWVec_XYZW = np.dot( headTransform_4x4,cycEyeInHead_XYZW)

# Now, convert into a direction from the cyclopean eye in world coordinates
# Also, we can discard the w term
cycGIWDir_XYZ = (cycGIWVec_XYZW[0:3]-rowIn["viewPos"]) / np.linalg.norm((cycGIWVec_XYZW[0:3]-rowIn["viewPos"]))

# You must return as a list or a tuple
#return list(cycGIWDir_XYZ)
return {('cycGIWDir','X'): cycGIWDir_XYZ[0],('cycGIWDir','Y'): cycGIWDir_XYZ[1],('cycGIWDir','Z'): cycGIWDir_XYZ[2]}

NameError: name 'rowIn' is not defined

In [ ]:
def calcGIW(sessionDictIn):

    proc = sessionDictIn['processedExp']

    if( ('cycGIW' in proc.columns) is True ):
            print('cycGIW is already in the dataframe.')
            return

    def eihToGIW(rowIn):

        # Grab gransformation matrix
        #headTransform_4x4 = np.reshape(rowIn["viewMat"],[4,4])

        mat =[]
        for i in range(16):
            mat.append(rowIn["camera"][str(i)])

        headTransform_4x4 = np.reshape(mat,[4,4])

        # Transpose
        headTransform_4x4 = headTransform_4x4.T

        # Grab cyc EIH direction
        cycEyeInHead_XYZ = rowIn['cycEyeInHead']
        # Add a 1 to convert to homogeneous coordinates
        cycEyeInHead_XYZW = np.hstack( [cycEyeInHead_XYZ,1])

        # Take the dot product!
        cycGIWVec_XYZW = np.dot( headTransform_4x4,cycEyeInHead_XYZW)

        # Now, convert into a direction from the cyclopean eye in world coordinates
        # Also, we can discard the w term
        cycGIWDir_XYZ = (cycGIWVec_XYZW[0:3]-rowIn["viewPos"]) / np.linalg.norm((cycGIWVec_XYZW[0:3]-rowIn["viewPos"]))

        # You must return as a list or a tuple
        #return list(cycGIWDir_XYZ)
        return {('cycGIWDir','X'): cycGIWDir_XYZ[0],('cycGIWDir','Y'): cycGIWDir_XYZ[1],('cycGIWDir','Z'): cycGIWDir_XYZ[2]}

    cycGIW = proc.apply(lambda row: eihToGIW(row),axis=1)
    cycGIWDf = pd.DataFrame.from_records(cycGIW)

    # cycGIWDf.columns = pd.MultiIndex.from_tuples(cycGIWDf.columns)
    # sessionDictIn['processed'] = proc.combine_first(pd.DataFrame.from_records(cycGIWDf))

    sessionDictIn['processedExp'] = sessionDictIn['processedExp'].join(cycGIWDf)

    logger.info('Added sessionDict[\'processedExp\'][\'cycGIWDir\']')
    return sessionDictIn

In [ ]:
calcGIW(sessionDict)

In [ ]:
list(df2)

In [ ]:
def calcGazeAzEl(row):
    
    x = row[('gaze-point-3d','x')] 
    y = row[('gaze-point-3d','y')]
    z = row[('gaze-point-3d','z')]
    
    
    
#     x = row[('gaze-point-3d','x')] - row[('cameraPos','x')]
#     y = row[('gaze-point-3d','y')] - row[('cameraPos','y')]
#     z = row[('gaze-point-3d','z')] - row[('cameraPos','z')]
    
    az = np.rad2deg(np.arctan(np.divide(x,z)))
    el = np.rad2deg(np.arctan(np.divide(y,z)))
    
    #Note that the .apply requires you return a single data structure, 
    #so a SINGLE tuple is OK, but the seperate values for az/el are not OK.
    
    return (az,el) 


azEl = df2.apply(lambda arbitraryRowName: calcGazeAzEl(arbitraryRowName),axis=1)

df2['fromGazePoint3d_az'],df2['fromGazePoint3d_el'] = zip(*azEl)



In [ ]:
# yLim = [-100,100]

yLim=[-50,50]
width=800
height=600

startTimeS = df2['worldVidTimeSec'].iloc[0]
endTimeS = df2['worldVidTimeSec'].iloc[-2]

# I googled for, "python find first larger than"
startIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  startTimeS)
endIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  endTimeS)
print('Plotting from index: ' + str(startIndex) + ' to ' + str(endIndex))

colors_idx = ['rgb(0,204,204)','rgb(128,128,128)','rgb(204,0,0)','rgb(102,0,204)', 'rgb(150,50,50)', 'rgb(50,50,150)']

# from plotly.subplots import make_subplots
# fig = make_subplots(rows=2, cols=1,shared_xaxes=True)

traces = []

###

bEih_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ball_el'][startIndex:endIndex],
    name = 'ball_el',
    marker_color = colors_idx[2],
    mode='markers',
    marker_size = 8,
    opacity = 0.5)

bEih_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ball_az'][startIndex:endIndex],
    name = 'ball_az',
    marker_color = colors_idx[3],
    mode='markers',
    marker_size = 8,
    opacity = 0.5)


eih_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=-df2['gazeInWorldCam_el'][startIndex:endIndex],
    name = 'gazeInWorldCam_el',
    marker_color = colors_idx[0],
    mode='markers',
    marker_size = 8,
    opacity = 1)

eih_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['gazeInWorldCam_az'][startIndex:endIndex],
    name = 'gazeInWorldCam_az',
    marker_color = colors_idx[1],
    mode='markers',
    marker_size = 8,
    opacity = 1)

traces.append(eih_el)
traces.append(eih_az)


#####


fromGP3D_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=-df2['fromGazePoint3d_el'][startIndex:endIndex],
    name = 'fromGazePoint3d_el',
    marker_color = colors_idx[4],
    mode='markers',
    marker_size = 3,
    opacity = .5)

fromGP3D_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['fromGazePoint3d_az'][startIndex:endIndex],
    name = 'fromGazePoint3d_az',
    marker_color = colors_idx[5],
    mode='markers',
    marker_size = 3,
    opacity = .5)

traces.append(fromGP3D_el)
traces.append(fromGP3D_az)


layout = dict(
    dragmode= 'pan',
    title='Time Series with Rangeslider',
    width=width,
    height=height,
    yaxis=dict(range=yLim, title='angular position (degrees)'),
    xaxis=dict(
        rangeslider=dict(visible=True))
)


fig = dict(data=traces, layout=layout)

iplot(fig)



In [ ]:
# yLim = [-100,100]

yLim=[-50,50]
width=800
height=600

startTimeS = df2['worldVidTimeSec'].iloc[0]
endTimeS = df2['worldVidTimeSec'].iloc[-2]

# I googled for, "python find first larger than"
startIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  startTimeS)
endIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  endTimeS)
print('Plotting from index: ' + str(startIndex) + ' to ' + str(endIndex))

colors_idx = ['rgb(0,204,204)','rgb(128,128,128)','rgb(204,0,0)','rgb(102,0,204)', 'rgb(150,50,50)', 'rgb(50,50,150)']

# from plotly.subplots import make_subplots
# fig = make_subplots(rows=2, cols=1,shared_xaxes=True)

traces = []

###

bEih_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ball_el'][startIndex:endIndex],
    name = 'ball_el',
    marker_color = colors_idx[2],
    mode='markers',
    marker_size = 8,
    opacity = 0.5)

bEih_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ball_az'][startIndex:endIndex],
    name = 'ball_az',
    marker_color = colors_idx[3],
    mode='markers',
    marker_size = 8,
    opacity = 0.5)


eih_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=-df2['gazeInWorldCam_el'][startIndex:endIndex],
    name = 'gazeInWorldCam_el',
    marker_color = colors_idx[0],
    mode='markers',
    marker_size = 8,
    opacity = 1)

eih_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['gazeInWorldCam_az'][startIndex:endIndex],
    name = 'gazeInWorldCam_az',
    marker_color = colors_idx[1],
    mode='markers',
    marker_size = 8,
    opacity = 1)

traces.append(eih_el)
traces.append(eih_az)


#####


fromGP3D_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=-df2['fromGazePoint3d_el'][startIndex:endIndex],
    name = 'fromGazePoint3d_el',
    marker_color = colors_idx[4],
    mode='markers',
    marker_size = 3,
    opacity = .5)

fromGP3D_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['fromGazePoint3d_az'][startIndex:endIndex],
    name = 'fromGazePoint3d_az',
    marker_color = colors_idx[5],
    mode='markers',
    marker_size = 3,
    opacity = .5)

traces.append(fromGP3D_el)
traces.append(fromGP3D_az)


layout = dict(
    dragmode= 'pan',
    title='Time Series with Rangeslider',
    width=width,
    height=height,
    yaxis=dict(range=yLim, title='angular position (degrees)'),
    xaxis=dict(
        rangeslider=dict(visible=True))
)


fig = dict(data=traces, layout=layout)

iplot(fig)


